## Import Modules

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
from os import listdir
from os.path import isfile, join
import time
import math
from IPython.display import clear_output
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt
import torchvision.transforms as T
#from torchsampler import ImbalancedDatasetSampler
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print('cuda available: '+ str(torch.cuda.is_available()))
#from skimage.morphology import disk, binary_dilation

from models_avm import NestedUNet
from loss_fun_avm import compute_per_channel_dice, DiceLoss, FocalLoss
from tra_val_avm import train, validation
from data_loader_avm import Dataset

cuda available: True


## Data Path

In [2]:
path_tra1 = '/work/samhong833/Data_AVM/forUNetpp/1_tra/1'
list_tra1 = os.listdir(path_tra1)
for i in range(len(list_tra1)):
    list_tra1[i] = path_tra1+'/'+list_tra1[i]
    
list_tra = list_tra1
    
path_val1 = '/work/samhong833/Data_AVM/forUNetpp/2_val/1'
list_val1 = os.listdir(path_val1)
for i in range(len(list_val1)):
    list_val1[i] = path_val1+'/'+list_val1[i]
    
list_val = list_val1

path_ts1 = '/work/samhong833/Data_AVM/forUNetpp/3_ts/1'
list_ts1 = os.listdir(path_ts1)
for i in range(len(list_ts1)):
    list_ts1[i] = path_ts1+'/'+list_ts1[i]
    
list_ts = list_ts1

path_tra_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/1_tra'
path_val_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/2_val'
path_ts_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/3_ts'

## Data Loader

In [3]:
train_data = Dataset(list_tra,path_tra_lab_txt,rand_dilate=True,max_dilate_factor=10)
train_loader = torch.utils.data.DataLoader(
    train_data, 
    batch_size = 12,
    shuffle = True,   
)

val_data = Dataset(list_val,path_val_lab_txt,rand_dilate=True,max_dilate_factor=10)
val_loader = torch.utils.data.DataLoader(
    dataset = val_data,           
    batch_size = 4,                 
    shuffle = False,              
)

## Initialize Model and Optimization Parameters

In [4]:
# call model cuda for gpu
model = NestedUNet().cuda()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.95)

# loss function
kwargs = {"alpha": 0.5, "gamma": 3, "reduction": 'mean'}
criterion_FL = FocalLoss(**kwargs)
criterion_DICE = DiceLoss()
loss = [criterion_FL,criterion_DICE]

## Start to Train

In [ ]:
# Create Directory
path = '/work/samhong833/Models/Seg_dia10'

if os.path.isdir(path)==False:
    os.mkdir(path)
path = os.path.join(path,"train")
if os.path.isdir(path)==False:
    os.mkdir(path)       
filenum = glob.glob(path + "/exp*")
path = path + "/exp" + str(len(filenum)+1)
os.mkdir(path)

# Train the Model
epochs = 500 # The number of epochs

valloss = 0
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(path=path,
          model=model,
          loss=loss,
          optimizer=optimizer,
          dataloader=train_loader,
          epoch=epoch,
          scheduler=scheduler)
    print('-' * 89)
    vallossnew = validation(path=path,
          model=model,
          loss=loss,
          dataloader=val_loader,
          epoch=epoch)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
    print('-' * 89)
    
    scheduler.step()
    
    f2 = open(path + '/model_info.txt', 'a')
    if vallossnew<valloss or epoch ==1: 
        fname = path + '/best_val'  + '.tar'
        torch.save(model.state_dict(), fname)
        valloss = vallossnew
        f2.write('| best_val | epoch {:3d}| '.format(epoch)+'\r\n')        
    f2.close()
    

| epoch   1 |   100/  290 batches | lr 0.0001000 | ms/batch 132774.60 | loss_fl 0.006661 | loss_dice  0.18 | 
| epoch   1 |   200/  290 batches | lr 0.0001000 | ms/batch 130117.40 | loss_fl 0.004414 | loss_dice  0.16 | 
| epoch   1 |   290/  290 batches | lr 0.0001000 | ms/batch 112877.19 | loss_fl 0.003488 | loss_dice  0.14 | 
-----------------------------------------------------------------------------------------
| epoch   1 |   100/  118 batches | | ms/batch 25631.33 | val_loss_fl 0.001289 | val_loss_dice  0.11 | 
| epoch   1 |   118/  118 batches | | ms/batch 4355.71 | val_loss_fl 0.001289 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 405.76s
-----------------------------------------------------------------------------------------
| epoch   2 |   100/  290 batches | lr 0.0001000 | ms/batch 109800.26 | loss_fl 0.001047 | loss_dice  0.11 | 
| epoch   2 |   200/  290 batches | lr 0.0001000

| epoch  11 |   100/  290 batches | lr 0.0000950 | ms/batch 105497.98 | loss_fl 0.000228 | loss_dice  0.08 | 
| epoch  11 |   200/  290 batches | lr 0.0000950 | ms/batch 105649.80 | loss_fl 0.000234 | loss_dice  0.08 | 
| epoch  11 |   290/  290 batches | lr 0.0000950 | ms/batch 94866.98 | loss_fl 0.000235 | loss_dice  0.08 | 
-----------------------------------------------------------------------------------------
| epoch  11 |   100/  118 batches | | ms/batch 17006.43 | val_loss_fl 0.000340 | val_loss_dice  0.08 | 
| epoch  11 |   118/  118 batches | | ms/batch 3033.06 | val_loss_fl 0.000340 | val_loss_dice  0.08 | 
-----------------------------------------------------------------------------------------
| end of epoch  11 | time: 326.06s
-----------------------------------------------------------------------------------------
| epoch  12 |   100/  290 batches | lr 0.0000950 | ms/batch 106290.23 | loss_fl 0.000216 | loss_dice  0.08 | 
| epoch  12 |   200/  290 batches | lr 0.0000950 

| epoch  21 |   100/  290 batches | lr 0.0000902 | ms/batch 108178.87 | loss_fl 0.000168 | loss_dice  0.07 | 
| epoch  21 |   200/  290 batches | lr 0.0000902 | ms/batch 108498.27 | loss_fl 0.000170 | loss_dice  0.07 | 
| epoch  21 |   290/  290 batches | lr 0.0000902 | ms/batch 97304.16 | loss_fl 0.000169 | loss_dice  0.07 | 
-----------------------------------------------------------------------------------------
| epoch  21 |   100/  118 batches | | ms/batch 17305.12 | val_loss_fl 0.000386 | val_loss_dice  0.08 | 
| epoch  21 |   118/  118 batches | | ms/batch 3080.14 | val_loss_fl 0.000386 | val_loss_dice  0.08 | 
-----------------------------------------------------------------------------------------
| end of epoch  21 | time: 334.37s
-----------------------------------------------------------------------------------------
| epoch  22 |   100/  290 batches | lr 0.0000902 | ms/batch 108444.98 | loss_fl 0.000163 | loss_dice  0.07 | 
| epoch  22 |   200/  290 batches | lr 0.0000902 

| epoch  31 |   100/  290 batches | lr 0.0000857 | ms/batch 107925.34 | loss_fl 0.000124 | loss_dice  0.06 | 
| epoch  31 |   200/  290 batches | lr 0.0000857 | ms/batch 107947.53 | loss_fl 0.000128 | loss_dice  0.05 | 
| epoch  31 |   290/  290 batches | lr 0.0000857 | ms/batch 97280.99 | loss_fl 0.000129 | loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| epoch  31 |   100/  118 batches | | ms/batch 17256.26 | val_loss_fl 0.000530 | val_loss_dice  0.09 | 
| epoch  31 |   118/  118 batches | | ms/batch 3073.42 | val_loss_fl 0.000521 | val_loss_dice  0.08 | 
-----------------------------------------------------------------------------------------
| end of epoch  31 | time: 333.49s
-----------------------------------------------------------------------------------------
| epoch  32 |   100/  290 batches | lr 0.0000857 | ms/batch 108275.88 | loss_fl 0.000122 | loss_dice  0.05 | 
| epoch  32 |   200/  290 batches | lr 0.0000857 

| epoch  41 |   100/  290 batches | lr 0.0000815 | ms/batch 108285.44 | loss_fl 0.000100 | loss_dice  0.05 | 
| epoch  41 |   200/  290 batches | lr 0.0000815 | ms/batch 108356.39 | loss_fl 0.000099 | loss_dice  0.05 | 
| epoch  41 |   290/  290 batches | lr 0.0000815 | ms/batch 96427.09 | loss_fl 0.000100 | loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| epoch  41 |   100/  118 batches | | ms/batch 17553.32 | val_loss_fl 0.000740 | val_loss_dice  0.09 | 
| epoch  41 |   118/  118 batches | | ms/batch 3132.77 | val_loss_fl 0.000733 | val_loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| end of epoch  41 | time: 333.76s
-----------------------------------------------------------------------------------------
| epoch  42 |   100/  290 batches | lr 0.0000815 | ms/batch 107371.36 | loss_fl 0.000097 | loss_dice  0.04 | 
| epoch  42 |   200/  290 batches | lr 0.0000815 

...

| epoch 171 |   100/  290 batches | lr 0.0000418 | ms/batch 110278.54 | loss_fl 0.000019 | loss_dice  0.01 | 
| epoch 171 |   200/  290 batches | lr 0.0000418 | ms/batch 110718.58 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 171 |   290/  290 batches | lr 0.0000418 | ms/batch 97748.78 | loss_fl 0.000018 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 171 |   100/  118 batches | | ms/batch 17235.02 | val_loss_fl 0.002695 | val_loss_dice  0.09 | 
| epoch 171 |   118/  118 batches | | ms/batch 3070.98 | val_loss_fl 0.002673 | val_loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| end of epoch 171 | time: 339.06s
-----------------------------------------------------------------------------------------
| epoch 172 |   100/  290 batches | lr 0.0000418 | ms/batch 108612.76 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 172 |   200/  290 batches | lr 0.0000418 

| epoch 181 |   100/  290 batches | lr 0.0000397 | ms/batch 110654.18 | loss_fl 0.000017 | loss_dice  0.01 | 
| epoch 181 |   200/  290 batches | lr 0.0000397 | ms/batch 111764.88 | loss_fl 0.000017 | loss_dice  0.01 | 
| epoch 181 |   290/  290 batches | lr 0.0000397 | ms/batch 101045.19 | loss_fl 0.000017 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 181 |   100/  118 batches | | ms/batch 17479.50 | val_loss_fl 0.002662 | val_loss_dice  0.09 | 
| epoch 181 |   118/  118 batches | | ms/batch 3186.43 | val_loss_fl 0.002612 | val_loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| end of epoch 181 | time: 344.13s
-----------------------------------------------------------------------------------------
| epoch 182 |   100/  290 batches | lr 0.0000397 | ms/batch 110115.49 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 182 |   200/  290 batches | lr 0.0000397

| epoch 191 |   100/  290 batches | lr 0.0000377 | ms/batch 110376.84 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 191 |   200/  290 batches | lr 0.0000377 | ms/batch 112293.11 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 191 |   290/  290 batches | lr 0.0000377 | ms/batch 103472.41 | loss_fl 0.000016 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 191 |   100/  118 batches | | ms/batch 17628.71 | val_loss_fl 0.003004 | val_loss_dice  0.10 | 
| epoch 191 |   118/  118 batches | | ms/batch 3130.81 | val_loss_fl 0.002926 | val_loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| end of epoch 191 | time: 346.91s
-----------------------------------------------------------------------------------------
| epoch 192 |   100/  290 batches | lr 0.0000377 | ms/batch 109871.44 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 192 |   200/  290 batches | lr 0.0000377

| epoch 201 |   100/  290 batches | lr 0.0000358 | ms/batch 108884.31 | loss_fl 0.000015 | loss_dice  0.01 | 
| epoch 201 |   200/  290 batches | lr 0.0000358 | ms/batch 109362.31 | loss_fl 0.000015 | loss_dice  0.01 | 
| epoch 201 |   290/  290 batches | lr 0.0000358 | ms/batch 100530.53 | loss_fl 0.000014 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 201 |   100/  118 batches | | ms/batch 18319.55 | val_loss_fl 0.002859 | val_loss_dice  0.09 | 
| epoch 201 |   118/  118 batches | | ms/batch 3209.00 | val_loss_fl 0.002829 | val_loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| end of epoch 201 | time: 340.31s
-----------------------------------------------------------------------------------------
| epoch 202 |   100/  290 batches | lr 0.0000358 | ms/batch 109579.54 | loss_fl 0.000014 | loss_dice  0.01 | 
| epoch 202 |   200/  290 batches | lr 0.0000358

| epoch 211 |   100/  290 batches | lr 0.0000341 | ms/batch 109592.17 | loss_fl 0.000013 | loss_dice  0.01 | 
| epoch 211 |   200/  290 batches | lr 0.0000341 | ms/batch 110629.96 | loss_fl 0.000013 | loss_dice  0.01 | 
| epoch 211 |   290/  290 batches | lr 0.0000341 | ms/batch 98810.17 | loss_fl 0.000013 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 211 |   100/  118 batches | | ms/batch 18663.16 | val_loss_fl 0.003013 | val_loss_dice  0.09 | 
| epoch 211 |   118/  118 batches | | ms/batch 3337.42 | val_loss_fl 0.002980 | val_loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| end of epoch 211 | time: 341.04s
-----------------------------------------------------------------------------------------
| epoch 212 |   100/  290 batches | lr 0.0000341 | ms/batch 108507.13 | loss_fl 0.000013 | loss_dice  0.01 | 
| epoch 212 |   200/  290 batches | lr 0.0000341 

# Log ^